In [ ]:
!python -m spacy download ru_core_news_lg

In [2]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load("ru_core_news_lg")

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [72]:
text = '''Приказ
Для организации мероприятия "День заказчика" приказываю:
1. Провести обучение для повышения квалификации сотрудников с учетом изменений в налоговом законодательстве.
Завершить выполнение к 18 дек 22.
Контроль над выполнением приказа возложить на специалиста третей категории бухгалтерии Александрова А.С.
2. Подготовить балансовый отчет за период. Привести вышеизложенное поручение в исполнение к 17 ноя 2018 г..
Контроль над выполнением приказа возложить на специалиста первой категории бухгалтерии Миронову Т.П.
3. Обеспечить архивирование отчетов о командировках сотрудников.
Определить срок исполнения задачи вплоть до 03 декабря 2015 года.
За контроль и выполнение назначить ответственным специалиста второй категории бухгалтерии Смирнову О.А.'''

In [73]:

triplets = {}

doc = nlp(text)
for token in enumerate(doc):

    if token[1].pos_ == "VERB":
        verb = [[token, 0]]

        subj = []
        obj = []

        subj1 = []
        obj1 = []
        
        for t_verb in enumerate(doc):
            if (t_verb[1].dep_ == "conj" and t_verb[1].head.text == token[1].text):
                verb.append([t_verb, 0])

        for i in verb:
            if i[0][0] != 0 and doc[i[0][0] - 1].text == "не":
                i[1] = 1

        for tok_verb in verb:
            for token1 in enumerate(doc):
                if str(token1[1].head) == tok_verb[0][1].text and token1[1].dep_ == "nsubj":
                    if token1[0] != 0 and doc[token1[0] - 1].text == "не":
                        subj.append("не " + token1[1].text)
                    else:
                        subj.append(token1[1].text)
                    subj1.append(token1[1].text)

                if str(token1[1].head) == tok_verb[0][1].text and token1[1].dep_ == "obj":
                    if token1[0] != 0 and doc[token1[0] - 1].text == "не":
                        obj.append("не " + token1[1].text)
                    else:
                        obj.append(token1[1].text)
                    obj1.append(token1[1].text)

            for t in enumerate(doc):
                if t[1].dep_ == "conj":
                    if t[1].head.text in subj1:
                      if t[0] != 0 and doc[t[0] - 1].text == "не":
                        subj.append("не " + t[1].text)
                      else:
                        subj.append(t[1].text)
                    if t[1].head.text in obj1:
                      if t[0] != 0 and doc[t[0] - 1].text == "не":
                        obj.append("не " + t[1].text)
                      else:
                        obj.append(t[1].text)
            #print(tok_verb, subj, obj)
            for i in subj:
                for j in obj:
                    if tok_verb[1]:
                        triplets[hash(i) + hash("не " + tok_verb[0][1].text) + hash(j)] = [i,"не " + tok_verb[0][1].text,j]
                    else:
                        triplets[hash(i) + hash(tok_verb[0][1].text) + hash(j)] = [i, tok_verb[0][1].text, j]
                    

    if token[1].pos_ == "NOUN":
        for token2 in doc:
            if str(token2.head) == token[1].text and token2.dep_ == "appos":
                triplets[hash(token2.text) + hash("есть") + hash(token[1].text)] = [token2.text, "есть", token[1].text]

    if token[1].pos_ == "ADJ":
        for token3 in doc:
            if str(token[1].head) == token3.text and token3.pos_ == "NOUN":
                triplets[hash(token3.text) + hash("есть") + hash(token[1].text)] = [token3.text, "есть", token[1].text]

displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

for i in triplets.keys():
    print(triplets[i])


['День', 'есть', 'мероприятия']
['законодательстве', 'есть', 'налоговом']
['Александрова', 'есть', 'специалиста']
['Миронову', 'есть', 'специалиста']
['Смирнову', 'есть', 'специалиста']
['отчет', 'есть', 'балансовый']
['поручение', 'есть', 'вышеизложенное']
['категории', 'есть', 'первой']
['года', 'есть', '2015']
['О.А.', 'есть', 'контроль']
['категории', 'есть', 'второй']


In [69]:
doc = nlp("Кошка имеет шерсть и когти.")


for token in list(doc.sents)[0]:
  print("token:", token, ", token.pos_:", token.pos_, ", token.head:", token.head, ", token.text:", token.text, ", token.dep_:", token.dep_)

token: Кошка , token.pos_: NOUN , token.head: имеет , token.text: Кошка , token.dep_: nsubj
token: имеет , token.pos_: VERB , token.head: имеет , token.text: имеет , token.dep_: ROOT
token: шерсть , token.pos_: NOUN , token.head: имеет , token.text: шерсть , token.dep_: obj
token: и , token.pos_: CCONJ , token.head: когти , token.text: и , token.dep_: cc
token: когти , token.pos_: NOUN , token.head: шерсть , token.text: когти , token.dep_: conj
token: . , token.pos_: PUNCT , token.head: имеет , token.text: . , token.dep_: punct


In [ ]:
sub = []
ob = []
pr = []

for i in triplets.keys():
  a = triplets[i]
  sub.append(a[0])
  pr.append(a[1])
  ob.append(a[2])

In [ ]:
import networkx as nx

kg_df = pd.DataFrame({'source':sub, 'target':ob, 'edge':pr})



G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='red', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df.loc[((kg_df['edge'] !="есть") | (kg_df['source']!="что") | (kg_df['source']!="Что"))], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes
nx.draw(G, with_labels=True, node_color='red', node_size=20, edge_cmap=plt.cm.Blues, pos = pos, font_weight='bold')
plt.show()